In [12]:
class Kakuro:
    WALL = 'X'
    EMPTY = ' '

    def __init__(self, field):
        self.field = field

    def get_segments(self, row, col):
        vertical_list = []
        horizontal_list = []
        vertical_target = None
        horizontal_target = None

        for r in range(row - 1, -1, -1):
            if self.field[r, col] == Kakuro.WALL or (isinstance(self.field[r, col], dict)):
                if isinstance(self.field[r, col], dict) and self.field[r, col]['top'] is not None:
                    vertical_target = self.field[r, col]['top']
                break
            vertical_list.append(self.field[r, col])

        for r in range(row + 1, len(self.field)):
            if self.field[r, col] == Kakuro.WALL or (isinstance(self.field[r, col], dict)):
                break
            vertical_list.append(self.field[r, col])

        for c in range(col - 1, -1, -1):
            if self.field[row, c] == Kakuro.WALL or (isinstance(self.field[row, c], dict)):
                if isinstance(self.field[row, c], dict) and self.field[row, c]['left'] is not None:
                    horizontal_target = self.field[row, c]['left']
                break
            horizontal_list.append(self.field[row, c])

        for c in range(col + 1, len(self.field[row])):
            if self.field[row, c] == Kakuro.WALL or (isinstance(self.field[row, c], dict)):
                break
            horizontal_list.append(self.field[row, c])

        return vertical_list, vertical_target, horizontal_list, horizontal_target

    def is_valid(self, row, col, num):
        vertical_list, vertical_target, horizontal_list, horizontal_target = self.get_segments(
            row, col)
        vertical_sum, horizontal_sum = 0, 0
        for item in vertical_list:
            if type(item) == int:
                vertical_sum += item

        for item in horizontal_list:
            if type(item) == int:
                horizontal_sum += item

        if vertical_target and (
            (num in vertical_list)
            or (vertical_list.count(Kakuro.EMPTY) == 0 and vertical_sum + num != vertical_target)
            or (vertical_sum + num > vertical_target)
        ):
            return False
        if horizontal_target and (
            (num in horizontal_list)
            or (horizontal_list.count(Kakuro.EMPTY) == 0 and horizontal_sum + num != horizontal_target)
            or (horizontal_sum + num > horizontal_target)
        ):
            return False

        return True

    def find_empty(self):
        for i in range(len(self.field)):
            for j in range(len(self.field[i])):
                if self.field[i, j] == Kakuro.EMPTY:
                    return i, j
        return None

    def solve(self):
        find = self.find_empty()
        if not find:
            return True
        else:
            row, col = find

        for i in range(1, 10):
            if self.is_valid(row, col, i):
                self.field[row][col] = i

                if self.solve():
                    return True

                self.field[row][col] = Kakuro.EMPTY

        return False

    @staticmethod
    def set_targets(left=None, top=None):
        return {
            'left': left,
            'top': top,
        }

    def __str__(self):
      table = ''
      for row in self.field:
          table += '|'
          for cell in row:
              if cell == Kakuro.WALL:
                  table += '  X  |'
              elif isinstance(cell, dict):
                  if cell.get('left') is not None:
                      table += f"  \\{cell['left']}  |"
                  elif cell.get('top') is not None:
                      table += f"  {cell['top']}\\  |"
              elif cell == 'B':
                  index_of_B = self.field[row, :].tolist().index(cell) + 1
                  table += f"  \\{index_of_B}  |"
              else:
                  table += f"  {cell}  |"
          table += '\n'
      return table



In [13]:
class KakuroOptimized(Kakuro):
    def find_most_constrained_empty(self):
        min_legal_values = 10
        most_constrained_cell = None

        for i in range(len(self.field)):
            for j in range(len(self.field[i])):
                if self.field[i, j] == Kakuro.EMPTY:
                    legal_values_count = self.count_legal_values(i, j)
                    if legal_values_count < min_legal_values:
                        min_legal_values = legal_values_count
                        most_constrained_cell = (i, j)

        return most_constrained_cell

    def count_legal_values(self, row, col):
        count = 0
        for value in range(1, 10):
            if self.is_valid(row, col, value):
                count += 1
        return count

    def get_legal_values(self, row, col):
        legal_values = []
        for value in range(1, 10):
            if self.is_valid(row, col, value):
                legal_values.append(value)
        return legal_values

    def solve(self):
        find = self.find_most_constrained_empty()
        if not find:
            return True
        else:
            row, col = find

        legal_values = self.get_legal_values(row, col)
        legal_values_sorted = self.sort_by_least_constraining_value(legal_values, row, col)

        for i in legal_values_sorted:
            self.field[row][col] = i
            if self.solve():
                return True
            self.field[row][col] = Kakuro.EMPTY

        return False

    def sort_by_least_constraining_value(self, legal_values, row, col):
        constraint_count = []

        for value in legal_values:
            count = self.count_constraints_for_value(row, col, value)
            constraint_count.append((value, count))

        constraint_count.sort(key=lambda x: x[1])
        sorted_values = [val for val, count in constraint_count]

        return sorted_values

    def count_constraints_for_value(self, row, col, value):
        count = 0

        original_value = self.field[row, col]
        self.field[row, col] = value

        for i in range(len(self.field)):
            for j in range(len(self.field[i])):
                if (i != row or j != col) and self.field[i, j] == Kakuro.EMPTY:
                    if not self.get_legal_values(i, j):
                        count += 1

        self.field[row, col] = original_value

        return count


In [14]:
import numpy as np
import time

In [15]:
def create_field(difficulty):
    if difficulty == 'easy':
        return np.array([
            [Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(top=30), Kakuro.set_targets(top=4), Kakuro.set_targets(top=24), Kakuro.WALL, Kakuro.set_targets(top=4), Kakuro.set_targets(top=16),],
            [Kakuro.WALL, Kakuro.set_targets(left=19, top=16), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=10, top=9), Kakuro.EMPTY, Kakuro.EMPTY,],
            [Kakuro.set_targets(left=39), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY,],
            [Kakuro.set_targets(left=15), Kakuro.EMPTY, Kakuro.EMPTY,  Kakuro.set_targets(left=10, top=23), Kakuro.EMPTY, Kakuro.EMPTY,  Kakuro.set_targets(top=10), Kakuro.WALL,],
            [Kakuro.WALL, Kakuro.set_targets(left=16), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=4, top=6), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=16),],
            [Kakuro.WALL, Kakuro.set_targets(top=14), Kakuro.set_targets(top=16,left=9), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=4, left=12), Kakuro.EMPTY, Kakuro.EMPTY,],
            [Kakuro.set_targets(left=35), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY,],
            [Kakuro.set_targets(left=16), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=7), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL,],
        ])
    elif difficulty == 'medium':
        return np.array([
            [Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(top=20), Kakuro.set_targets(top=3), Kakuro.set_targets(top=23), Kakuro.WALL, Kakuro.set_targets(top=12), Kakuro.set_targets(top=16)],
            [Kakuro.WALL, Kakuro.set_targets(left=12, top=5), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=16, top=24), Kakuro.EMPTY, Kakuro.EMPTY],
            [Kakuro.set_targets(left=41), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY],
            [Kakuro.set_targets(left=3), Kakuro.EMPTY, Kakuro.EMPTY , Kakuro.set_targets(left=13, top=24), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=11), Kakuro.WALL],
            [Kakuro.WALL, Kakuro.set_targets(left=17), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=10, top=23), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=16)],
            [Kakuro.WALL, Kakuro.set_targets(top=14), Kakuro.set_targets(left=16, top=5), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=11, top=17), Kakuro.EMPTY, Kakuro.EMPTY],
            [Kakuro.set_targets(left=42), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY],
            [Kakuro.set_targets(left=10), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=22) ,Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL],
        ])
    elif difficulty == 'hard':
        return np.array([
            [Kakuro.WALL, Kakuro.set_targets(top=10), Kakuro.set_targets(top=10), Kakuro.WALL, Kakuro.WALL, Kakuro.WALL, Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(top=23), Kakuro.set_targets(top=16)],
            [Kakuro.set_targets(left=4), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=17), Kakuro.WALL, Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(left=16, top=17), Kakuro.EMPTY, Kakuro.EMPTY],
            [Kakuro.set_targets(left=23), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=20), Kakuro.WALL, Kakuro.set_targets(left=24, top= 30), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY],
            [Kakuro.WALL, Kakuro.set_targets(left=13), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=23, top=20), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL],
            [Kakuro.WALL, Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(left=11), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL, Kakuro.WALL],
            [Kakuro.WALL, Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(left=23, top=6), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL, Kakuro.WALL, Kakuro.WALL],
            [Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(left=25, top=7), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=3), Kakuro.set_targets(top=9), Kakuro.WALL],
            [Kakuro.WALL, Kakuro.set_targets(left=8, top=4), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=7), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=4)],
            [Kakuro.set_targets(left=6), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(left=6), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY],
            [Kakuro.set_targets(left=3), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL, Kakuro.WALL, Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(left=4), Kakuro.EMPTY, Kakuro.EMPTY],
        ])
    elif difficulty == 'expert':
        return  np.array([
            [Kakuro.WALL, Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(top=17), Kakuro.set_targets(top=19), Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(top=7), Kakuro.set_targets(top=44), Kakuro.WALL],
            [Kakuro.WALL, Kakuro.set_targets(top=3), Kakuro.set_targets(top=37, left=17), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL, Kakuro.set_targets(left=10), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=23)],
            [Kakuro.set_targets(left=20), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=6), Kakuro.set_targets(top=3, left=15), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY],
            [Kakuro.set_targets(left=5), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=25, top=3), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY],
            [Kakuro.WALL, Kakuro.set_targets(left=8), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=3), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=15,top=10), Kakuro.EMPTY, Kakuro.EMPTY],
            [Kakuro.WALL, Kakuro.set_targets(left=3, top=13), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=7), Kakuro.set_targets(top=5), Kakuro.set_targets(left=17), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL],
            [Kakuro.set_targets(left=9), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=3, top=10), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=6,top=16), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=11)],
            [Kakuro.set_targets(left=38), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=17, top=3), Kakuro.EMPTY, Kakuro.EMPTY],
            [Kakuro.set_targets(left=7), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL, Kakuro.set_targets(left=12), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY],
            [Kakuro.WALL, Kakuro.set_targets(left=4), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL, Kakuro.set_targets(left=3), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL, Kakuro.WALL]
        ])


In [16]:
def solve_and_measure(difficulty):
    # Create Kakuro instances for both standard and optimized versions
    field_standard = create_field(difficulty)
    game_standard = Kakuro(field_standard)

    field_optimized = create_field(difficulty)
    game_optimized = KakuroOptimized(field_optimized)

    # Measure time for standard Kakuro
    start_time = time.time()
    game_standard.solve()
    standard_time = time.time() - start_time

    # Measure time for optimized Kakuro
    start_time = time.time()
    game_optimized.solve()
    optimized_time = time.time() - start_time

    return standard_time, optimized_time


In [18]:
for difficulty in ['easy', 'medium', 'hard', 'expert']:
    # for diff in []
    standard_time, optimized_time = solve_and_measure(difficulty)

    # Print results
    print(f"{difficulty.title()} Difficulty:")
    print(f"Backtrack Kakuro solve: {standard_time:.4f} seconds")
    print(f"Optimized Kakuro solve: {optimized_time:.4f} seconds")

    # Print the original table
    print("\nOriginal Kakuro Table:")
    field = create_field(difficulty)
    print(Kakuro(field))

    # Print the optimized table
    print("\nOptimized Kakuro Solved:")
    field_optimized = create_field(difficulty)
    game_optimized = KakuroOptimized(field_optimized)
    game_optimized.solve()
    print(game_optimized)

    print("----------")

Easy Difficulty:
Standard Kakuro: 0.8673 seconds
Optimized Kakuro: 0.3596 seconds

Original Kakuro Table:
|  X  |  X  |  30\  |  4\  |  24\  |  X  |  4\  |  16\  |
|  X  |  \19  |     |     |     |  \10  |     |     |
|  \39  |     |     |     |     |     |     |     |
|  \15  |     |     |  \10  |     |     |  10\  |  X  |
|  X  |  \16  |     |     |  \4  |     |     |  16\  |
|  X  |  14\  |  \9  |     |     |  \12  |     |     |
|  \35  |     |     |     |     |     |     |     |
|  \16  |     |     |  \7  |     |     |     |  X  |


Optimized Kakuro Solved:
|  X  |  X  |  30\  |  4\  |  24\  |  X  |  4\  |  16\  |
|  X  |  \19  |  9  |  3  |  7  |  \10  |  1  |  9  |
|  \39  |  9  |  6  |  1  |  8  |  5  |  3  |  7  |
|  \15  |  7  |  8  |  \10  |  9  |  1  |  10\  |  X  |
|  X  |  \16  |  7  |  9  |  \4  |  3  |  1  |  16\  |
|  X  |  14\  |  \9  |  6  |  3  |  \12  |  3  |  9  |
|  \35  |  5  |  9  |  8  |  1  |  3  |  2  |  7  |
|  \16  |  9  |  7  |  \7  |  2  |  1  |  4  |  X 

## other tabels

In [ ]:
# def create_field(difficulty):
#     if difficulty == 'easy':
#         field1 = np.array([
#             [Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(top=30), Kakuro.set_targets(top=4), Kakuro.set_targets(
#                 top=24), Kakuro.WALL, Kakuro.set_targets(top=4), Kakuro.set_targets(top=16),],
#             [Kakuro.WALL, Kakuro.set_targets(left=19, top=16), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(
#                 left=10, top=9), Kakuro.EMPTY, Kakuro.EMPTY,],
#             [Kakuro.set_targets(left=39), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY,
#              Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY,],
#             [Kakuro.set_targets(left=15), Kakuro.EMPTY, Kakuro.EMPTY,  Kakuro.set_targets(
#                 left=10, top=23), Kakuro.EMPTY, Kakuro.EMPTY,  Kakuro.set_targets(top=10), Kakuro.WALL,],
#             [Kakuro.WALL, Kakuro.set_targets(left=16), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(
#                 left=4, top=6), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=16),],
#             [Kakuro.WALL, Kakuro.set_targets(top=14), Kakuro.set_targets(
#                 top=16, left=9), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=4, left=12), Kakuro.EMPTY, Kakuro.EMPTY,],
#             [Kakuro.set_targets(left=35), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY,
#              Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY,],
#             [Kakuro.set_targets(left=16), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(
#                 left=7), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL,],
#         ])
        
#         field2= np.array([
#             [Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(top=30), Kakuro.set_targets(top=4), Kakuro.set_targets(top=24), Kakuro.WALL, Kakuro.set_targets(top=4), Kakuro.set_targets(top=16)],
#             [Kakuro.WALL, Kakuro.set_targets(left=19, top=16), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=10, top=9), Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.set_targets(left=39), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.set_targets(left=15), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=10, top=23), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=10), Kakuro.WALL],
#             [Kakuro.WALL, Kakuro.set_targets(left=16), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=4, top=6), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=16)],
#             [Kakuro.WALL, Kakuro.set_targets(top=14), Kakuro.set_targets(top=16, left=9), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=4, left=12), Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.set_targets(left=35), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.set_targets(left=16), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=7), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL],
#         ])
#         field3 =np.array([
#             [Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(top=20), Kakuro.set_targets(top=3), Kakuro.set_targets(top=23), Kakuro.WALL, Kakuro.set_targets(top=12), Kakuro.set_targets(top=16)],
#             [Kakuro.WALL, Kakuro.set_targets(left=12, top=5), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=16, top=24), Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.set_targets(left=41), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.set_targets(left=3), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=13, top=24), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=11), Kakuro.WALL],
#             [Kakuro.WALL, Kakuro.set_targets(left=17), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=10, top=23), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=16)],
#             [Kakuro.WALL, Kakuro.set_targets(top=14), Kakuro.set_targets(left=16, top=5), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=11, top=17), Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.set_targets(left=42), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.set_targets(left=10), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=22), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL],
#         ])
        
#         return field1, field2, field3
    
#     elif difficulty == 'medium':
#         field1= np.array([
#             [Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(top=20), Kakuro.set_targets(top=3), Kakuro.set_targets(
#                 top=23), Kakuro.WALL, Kakuro.set_targets(top=12), Kakuro.set_targets(top=16)],
#             [Kakuro.WALL, Kakuro.set_targets(left=12, top=5), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(
#                 left=16, top=24), Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.set_targets(left=41), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY,
#              Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.set_targets(left=3), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(
#                 left=13, top=24), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=11), Kakuro.WALL],
#             [Kakuro.WALL, Kakuro.set_targets(left=17), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(
#                 left=10, top=23), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=16)],
#             [Kakuro.WALL, Kakuro.set_targets(top=14), Kakuro.set_targets(
#                 left=16, top=5), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=11, top=17), Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.set_targets(left=42), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY,
#              Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.set_targets(left=10), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(
#                 left=22), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL],
#         ])
        
#         field2 = np.array([
#             [Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(top=20), Kakuro.set_targets(top=3), Kakuro.set_targets(top=23), Kakuro.WALL, Kakuro.set_targets(top=12), Kakuro.set_targets(top=16)],
#             [Kakuro.WALL, Kakuro.set_targets(left=12, top=5), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=16, top=24), Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.set_targets(left=41), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.set_targets(left=3), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=13, top=24), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=11), Kakuro.WALL],
#             [Kakuro.WALL, Kakuro.set_targets(left=17), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=10, top=23), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=16)],
#             [Kakuro.WALL, Kakuro.set_targets(top=14), Kakuro.set_targets(left=16, top=5), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=11, top=17), Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.set_targets(left=42), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.set_targets(left=10), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=22), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL],
#         ])
#         field3 = np.array([
#             [Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(top=30), Kakuro.set_targets(top=4), Kakuro.set_targets(top=24), Kakuro.WALL, Kakuro.set_targets(top=4), Kakuro.set_targets(top=16)],
#             [Kakuro.WALL, Kakuro.set_targets(left=19, top=16), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=10, top=9), Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.set_targets(left=39), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.set_targets(left=15), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=10, top=23), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=10), Kakuro.WALL],
#             [Kakuro.WALL, Kakuro.set_targets(left=16), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=4, top=6), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=16)],
#             [Kakuro.WALL, Kakuro.set_targets(top=14), Kakuro.set_targets(top=16, left=9), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=4, left=12), Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.set_targets(left=35), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.set_targets(left=16), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=7), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL],
#         ])
        
#         return field1, field2, field3
#     elif difficulty == 'hard':
#         field1= np.array([
#             [Kakuro.WALL, Kakuro.set_targets(top=10), Kakuro.set_targets(
#                 top=10), Kakuro.WALL, Kakuro.WALL, Kakuro.WALL, Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(top=23), Kakuro.set_targets(top=16)],
#             [Kakuro.set_targets(left=4), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(
#                 top=17), Kakuro.WALL, Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(left=16, top=17), Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.set_targets(left=23), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(
#                 top=20), Kakuro.WALL, Kakuro.set_targets(left=24, top=30), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.WALL, Kakuro.set_targets(left=13), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(
#                 left=23, top=20), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL],
#             [Kakuro.WALL, Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(
#                 left=11), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL, Kakuro.WALL],
#             [Kakuro.WALL, Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(
#                 left=23, top=6), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL, Kakuro.WALL, Kakuro.WALL],
#             [Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(left=25, top=7), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(
#                 top=3), Kakuro.set_targets(top=9), Kakuro.WALL],
#             [Kakuro.WALL, Kakuro.set_targets(left=8, top=4), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(
#                 left=7), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=4)],
#             [Kakuro.set_targets(left=6), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL,
#              Kakuro.WALL, Kakuro.set_targets(left=6), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.set_targets(left=3), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL, Kakuro.WALL,
#              Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(left=4), Kakuro.EMPTY, Kakuro.EMPTY],
#         ])
#         field2 = np.array([
#             [Kakuro.WALL, Kakuro.set_targets(top=10), Kakuro.set_targets(top=10), Kakuro.WALL, Kakuro.WALL, Kakuro.WALL, Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(top=23), Kakuro.set_targets(top=16)],
#             [Kakuro.set_targets(left=4), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=17), Kakuro.WALL, Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(left=16, top=17), Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.set_targets(left=23), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=20), Kakuro.WALL, Kakuro.set_targets(left=24, top=30), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.WALL, Kakuro.set_targets(left=13), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=23, top=20), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL],
#             [Kakuro.WALL, Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(left=11), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL, Kakuro.WALL],
#             [Kakuro.WALL, Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(left=23, top=6), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL, Kakuro.WALL, Kakuro.WALL],
#             [Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(left=25, top=7), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=3), Kakuro.set_targets(top=9), Kakuro.WALL],
#             [Kakuro.WALL, Kakuro.set_targets(left=8, top=4), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=7), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=4)],
#             [Kakuro.set_targets(left=6), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(left=6), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.set_targets(left=3), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL, Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(left=4), Kakuro.EMPTY, Kakuro.EMPTY],
#         ])
#         field3 = np.array([
#             [Kakuro.WALL, Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(top=15), Kakuro.set_targets(top=20), Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(top=16), Kakuro.set_targets(top=17), Kakuro.WALL],
#             [Kakuro.WALL, Kakuro.set_targets(top=7), Kakuro.set_targets(top=21, left=16), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL, Kakuro.set_targets(left=10), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=12)],
#             [Kakuro.set_targets(left=22), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=6), Kakuro.set_targets(top=3, left=15), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.set_targets(left=5), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=25, top=3), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.WALL, Kakuro.set_targets(left=8), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=3), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=15, top=10), Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.WALL, Kakuro.set_targets(left=3, top=13), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=7), Kakuro.set_targets(top=5), Kakuro.set_targets(left=17), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL],
#             [Kakuro.set_targets(left=9), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=3, top=10), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=6, top=16), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=11)],
#             [Kakuro.set_targets(left=38), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=17, top=3), Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.set_targets(left=7), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL, Kakuro.set_targets(left=12), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.WALL, Kakuro.set_targets(left=4), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL, Kakuro.set_targets(left=3), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL, Kakuro.WALL],
#         ])
        
#         return field1, field2, field3
#     elif difficulty == 'expert':
#         field1= np.array([
#             [Kakuro.WALL, Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(top=17), Kakuro.set_targets(
#                 top=19), Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(top=7), Kakuro.set_targets(top=44), Kakuro.WALL],
#             [Kakuro.WALL, Kakuro.set_targets(top=3), Kakuro.set_targets(top=37, left=17), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL, Kakuro.set_targets(
#                 left=10), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=23)],
#             [Kakuro.set_targets(left=20), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(
#                 top=6), Kakuro.set_targets(top=3, left=15), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.set_targets(left=5), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(
#                 left=25, top=3), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.WALL, Kakuro.set_targets(left=8), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(
#                 left=3), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=15, top=10), Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.WALL, Kakuro.set_targets(left=3, top=13), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(
#                 top=7), Kakuro.set_targets(top=5), Kakuro.set_targets(left=17), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL],
#             [Kakuro.set_targets(left=9), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=3, top=10), Kakuro.EMPTY,
#              Kakuro.EMPTY, Kakuro.set_targets(left=6, top=16), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=11)],
#             [Kakuro.set_targets(left=38), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY,
#              Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=17, top=3), Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.set_targets(left=7), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL, Kakuro.set_targets(
#                 left=12), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.WALL, Kakuro.set_targets(left=4), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL, Kakuro.set_targets(
#                 left=3), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL, Kakuro.WALL]
#         ])
#         field2 = np.array([
#             [Kakuro.WALL, Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(top=17), Kakuro.set_targets(top=19), Kakuro.WALL, Kakuro.WALL, Kakuro.set_targets(top=7), Kakuro.set_targets(top=44), Kakuro.WALL],
#             [Kakuro.WALL, Kakuro.set_targets(top=3), Kakuro.set_targets(top=37, left=17), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL, Kakuro.set_targets(left=10), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=23)],
#             [Kakuro.set_targets(left=20), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=6), Kakuro.set_targets(top=3, left=15), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.set_targets(left=5), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=25, top=3), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.WALL, Kakuro.set_targets(left=8), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=3), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=15,top=10), Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.WALL, Kakuro.set_targets(left=3, top=13), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=7), Kakuro.set_targets(top=5), Kakuro.set_targets(left=17), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL],
#             [Kakuro.set_targets(left=9), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=3, top=10), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=6,top=16), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(top=11)],
#             [Kakuro.set_targets(left=38), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.set_targets(left=17, top=3), Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.set_targets(left=7), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL, Kakuro.set_targets(left=12), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.EMPTY],
#             [Kakuro.WALL, Kakuro.set_targets(left=4), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL, Kakuro.set_targets(left=3), Kakuro.EMPTY, Kakuro.EMPTY, Kakuro.WALL, Kakuro.WALL]
#         ])
#         # field3 = 
        
#         return field1, field2